In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
from functools import partial
import itertools
import os
import json
from decimal import Decimal
import time
from scipy.integrate import solve_ivp
import pandas as pd
import traceback
import copy
import concurrent.futures
import multiprocessing as mp
import sys
import uuid
import gc

from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)


%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams.update({'font.size': 13})

In [ ]:
def sci_notation(number, sig_fig=2):
    form_string = "{0:.{1:d}e}".format(number, sig_fig)
    a,b = form_string.split("e")
    b = int(b)
    c = r"$10^{"+str(b)+r"}$"
    if float(number).is_integer() and number <= 10:
        return str(int(number))
    elif a == '1.00':
        return c
    elif float(a).is_integer():
        return str(int(float(a))) + r"$\cdot$" + c
    elif b == 0:
        return a
    else:
        return a + r"$\cdot$" + c

In [ ]:
def axes_tick_formatter(self, MajorTicks, MinorTicks, axis = 'x'):
    if axis == 'x':
        self.xaxis.set_major_locator(MultipleLocator(MajorTicks))
        self.xaxis.set_minor_locator(MultipleLocator(MinorTicks))
    elif axis == 'y':
        self.yaxis.set_major_locator(MultipleLocator(MajorTicks))
        self.yaxis.set_minor_locator(MultipleLocator(MinorTicks))
    else:
        print("Algo mal hay")

In [ ]:
def remove_zeros(a):
    
    if a[-1] != '0' and a[-1] != '.':
        return str(a)   
        
    else:
        c = str(a[:-1])
        return(remove_zeros(c))

def param_folder_notation(number, sig_fig=2):
    if number == 0.0:
        c = '0'
    
    else:    
        form_string = "{0:.{1:d}e}".format(number, sig_fig)
        a,b = form_string.split("e")
        a = remove_zeros(a)
        b = int(b)

        if float(a).is_integer():
            if b == 0:
                c = str(int(float(a)))
            else:
                c = str(int(float(a))) +'e'+ str(b)
        else:
            if b == 0:
                c = a
            else:
                c = a +'e'+ str(b)
        c = c.replace('-','m').replace('.','p')
    
    return c

In [ ]:
def m_notch_epsin_coop(t, x, params):
    bl = params["bl"]
    dl = params["dl"]
    
    kr = params["kr"]
    krl = params["krl"]
    
    br = params["br"]
    dr = params["dr"]
    
    kb = params["kb"]
    ku = params["ku"]
    
    km = params["km"]
    n = params["n"]
    m = params["m"]
    
    
    ks2 = params["ks2"]
    kp = params["kp"]
    
    ki = params["ki"]
    ks3 = params["ks3"]
    ds = params["ds"]
    
    KG = params["KG"]
    TN = params["TN"]
    CN = params["CN"]
    
    dG = params["dG"]
    dm = params["dm"]
    
    R = x[0]
    L = x[1]
    E = x[2]
    C = x[3]
    CL = x[4]
    CLi = x[5]
    CR = x[6]
    S = x[7]
    G4 = x[8]
    Cit = x[9]

    
    Dx = np.array([
        br - dr * R - kb * (R * L) + ku * C, #R
        bl - dl * L - kb * (R * L) + ku * C + n * krl * kr * CLi , #L
        m * kr * CLi - m * ks2 * km ** (1-m) * C ** n * E ** m , #E
#         0., #E
        kb * R * L - ku * C - kp * C - n * ks2 * km ** (1-m) * C ** n * E ** m, #C
        ks2 * km ** (1-m) * C ** n * E ** m - ki * CL, #CL
        ki * CL - kr * CLi, #CLi
        ks2 * km ** (1-m) * C ** n * E ** m - ks3 * CR, #CR
        n * ks3 * CR - ds * S, #S
        n * ks3 * CR - dG * G4, #G4
        CN * TN * G4 / (G4 + KG) - dm * Cit #Cit
        
    ])
    
    return Dx

In [ ]:
def x_interpolation(r, q, y):
    if r[0] >= q[0]:
        p_0 = q
        p_1 = r
    else:
        p_0 = r
        p_1 = q

    x = p_0[0] + (y - p_0[1]) / (p_1[1] - p_0[1]) * (p_1[0] - p_0[0])

    return x

def peak_values (np_input_x, np_input_y, eps = 0.1):
    max_value = max(np_input_y)
    max_value_list = np.where(np_input_y == max_value)
    max_value_index = int(np.round(np.mean(max_value_list[0])))

    end_value = np_input_y[-1]

    t2 = 0
    t1 = 0

    if end_value < 0.5 * max_value:

        half_max = 0.5 * max_value

        mvi_left = max_value_index - 1
        mvi_right = max_value_index + 1

        while np_input_y[mvi_left] > half_max:
            mvi_left -= 1

        while np_input_y[mvi_right] > half_max:
            mvi_right += 1

        r1 = [np_input_x[mvi_left], np_input_y[mvi_left]]
        q1 = [np_input_x[mvi_left + 1], np_input_y[mvi_left + 1]]

        t1 = x_interpolation(r1, q1, half_max)

        r2 = [np_input_x[mvi_right], np_input_y[mvi_right]]
        q2 = [np_input_x[mvi_right - 1], np_input_y[mvi_right - 1]]

        t2 = x_interpolation(r2, q2, half_max)
        
    max_12 = 0.5 * max_value
    
    k = 0
    
    while np_input_y[k] < max_12:
        k += 1
        
    r12 = [np_input_x[k], np_input_y[k]]
    q12 = [np_input_x[k - 1], np_input_y[k - 1]]
    
    t12 = x_interpolation(r12, q12, max_12)
    
    t_max = np_input_x[max_value_index]

    return (t1, t2, max_value, end_value, t12, t_max)

def HS_f (a,b,eps):
    if a/b>eps:
        return 1.
    else:
        return 0.

In [ ]:
def f_RS (peak):
    max_val = peak[2]
    end_val = peak[3]
    
    H = max_val - end_val
    
    return (H/end_val)

def f_RSM (peak):
    max_val = peak[2]
    end_val = peak[3]
    
    H = max_val - end_val
    
    return (H/max_val)

def f_NRS (peak, params):
    br = params['br']
    ds = params['ds']
    
    max_val = peak[2]
    end_val = peak[3]
    
    H = max_val - end_val
    S0 = br/ds
    
    return (H/S0)

def f_DT (peak, eps):

    t1 = peak[0]
    t2 = peak[1]
    
    max_val = peak[2]
    end_val = peak[3]
    
    HS = HS_f(max_val-end_val, max_val, eps)
    
    return  (t2 - t1)

def f_RT (peak, params, eps):
    ds = params['ds']

    t1 = peak[0]
    t2 = peak[1]
    
    max_val = peak[2]
    end_val = peak[3]
    
    HS = HS_f(max_val-end_val, max_val, eps)
    
    return (HS * (t2 - t1) * ds)

def f_GT(peak, mu, sd):
    t1 = peak[0]
    t2 = peak[1]
    
    dt = t2-t1
    
    return np.exp(-((mu-dt)/sd)**2)

def f_GS(rs, mu, sd):
    return np.exp(-((mu-rs)/sd)**2)

def f_GT12(peak, mu, sd):
    t12 = peak[4]
    
    return np.exp(-((mu-t12)/sd)**2)

In [ ]:
def solve_resolution(KI, KJ, PPNAMES, PPVALUES, X0, PARAMS, variable, p_gauss, model = m_notch_epsin_coop, param_iter = [],
                    time_l = [0,25*60], method = 'Radau'):
    k_i = KI
    k_j = KJ
    p_p_names = PPNAMES
    p_p_values = PPVALUES
    x_0 = X0
    params = PARAMS
    
    p_p_values_iter = list(itertools.product(*p_p_values))
    
    if param_iter == []:
        param_iter = []
        param_iter_dic = []
        for  e in p_p_values_iter:
            param_i = dict(params)
            param_i.update(zip(p_p_names, e))
            param_iter.append(tuple([dict(param_i)]))
            param_iter_dic.append(dict(param_i))
            
    else:
        param_iter_dic = []
        for  e in param_iter:
            param_iter_dic.append(dict(e[0]))
            
    if type(x_0[0])!=list:
        partial_iter = partial(par_solve_ivp, model, time_l, x_0, method)
        sol = [partial_iter(e) for e in param_iter]
        
    else:
        partial_iter = partial(par_solve_ivp, model, time_l)
        sol = [partial_iter(f,method,e) for e,f in zip(param_iter,x_0)]
    
    T_H = [e.t / 60. for e in sol]
    
    res = [peak_values(T_H[e], sol[e].y[variable]) for e in range(len(sol))]
    s_max = [max(sol[e].y[variable]) for e in range(len(sol))]
    s_stat = [sol[e].y[variable][-1] for e in range(len(sol))]
    table_params = [[p_p_values_iter[j] for j in range(i*len(k_j),(i+1)*len(k_j))]for i in range(len(k_i))]
    table_params_dict = [[param_iter_dic[j] for j in range(i*len(k_j),(i+1)*len(k_j))]for i in range(len(k_i))]
    table_values =  [[res[j] for j in range(i*len(k_j),(i+1)*len(k_j))]for i in range(len(k_i))]
    SMax =  [[s_max[j] for j in range(i*len(k_j),(i+1)*len(k_j))]for i in range(len(k_i))]
    Sstat =  [[s_stat[j] for j in range(i*len(k_j),(i+1)*len(k_j))]for i in range(len(k_i))]
    

    len_i = len(table_values)
    len_j = len(table_values[0])
    
    mut = p_gauss["mut"]
    sdt = p_gauss["sdt"]
    
    mut12 = p_gauss["mut12"]
    sdt12 = p_gauss["sdt12"]

    mus = p_gauss["mus"]
    sds = p_gauss["sds"]

    eps = p_gauss["eps"]

    RSM = [[f_RSM(tvij) for tvij in tvi] for tvi in table_values]

    RT = [[f_RT(table_values[i][j], table_params_dict[i][j], eps) for j in
          range(len_j)] for i in range(len_i)]
    
    RM = [[RT[i][j] * RSM[i][j] for j in range(len_j)] for i in range(len_i)]

    GSM = [[f_GS(RSM[i][j], mus, sds) for j in range(len_j)] for i in
          range(len_i)]

    GT = [[f_GT(tvij, mut, sdt) for tvij in tvi] for tvi in table_values]

    GM = [[GT[i][j] * GSM[i][j] for j in range(len_j)] for i in range(len_i)]
    
    RT12 = [[tvij[4] for tvij in tvi] for tvi in table_values]
    
    GT12 = [[f_GT12(tvij, mut12, sdt12) for tvij in tvi] for tvi in table_values]
    
    DT = [[f_DT(tvij, eps) for tvij in tvi] for tvi in table_values]
    
    TMAX = [[tvij[5] for tvij in tvi] for tvi in table_values]
    
    return [RSM, RT, RM, GSM, GT, GM, RT12, GT12, SMax, Sstat, DT, TMAX]

In [ ]:
def is_zero(a):
    return(1 if int(a) == int(0) else 0)
is_zero_vec = np.vectorize(is_zero)

contour_values_index=[0]
contour_levels=[[0.7,0.8]]
contour_colours=[["#36454F","#808080"]]


In [ ]:
def rk4(f,x,t,dt,params):
    
    k1 = f(t, x, params)
    k2 = f(t, x + k1*dt/2, params)
    k3 = f(t, x + k2*dt/2, params)    
    k4 = f(t, x + k3*dt, params)
    
    k = dt * (k1 + 2*k2 + 2*k3 + k4)/6
    
    return (x + k)

In [ ]:
def par_solve_ivp(model, time_l, x_0, meth, arg):
    return solve_ivp(model, time_l, x_0, method = meth, args=arg)

In [ ]:

llss = ["-", ":", "-.", "--"]

variables = ["R","L","E","C",r"$C_L$",r"$C_{Li}$",r"$C_R$","S","Gal4","Cit"]
variables_f = ["R","L","E","C",r"C_L",r"C_Li",r"C_R","S","Gal4","Cit"]
ligands = ["Dll1", r"$Dll4_{ECD}$"+"\n"+r"$Dll1_{ICD}$", "Dll4", r"$Dll1_{ECD}$"+"\n"+r"$Dll4_{ICD}$"]
ligands_f = ["Dll1", "1I4E", "Dll4", "4I1E"]
cm_variables = ["#194569","#800020","#ff7f0e","#9467bd","#c20e35","#fa8fad","#1f77b4","#80bce6","#c8f1c8","#77dd77"]
cm_ligands = ["#194569","#ff7f0e","#800020","#9467bd"]
color_bars = [cm_variables[5],cm_variables[4],cm_variables[1]]


p_gauss_S = {"mut": 0.8, "sdt": 0.2, "mut12": 0.1, "sdt12":0.02, "mus": 0.9, "sds": 0.15, "eps": 0.1}
p_gauss_Citrine = {"mut": 9.5, "sdt": 4.5, "mut12": 1.5, "sdt12":0.5, "mus": 0.8, "sds": 0.15, "eps": 0.1}

range_vals = [[0.,1.], 1, [0.,1.], [0.,1.], [0.,1.], [0.,1.], 1, [0.,1.], 0, 0, 0]


In [ ]:
t12_G = 3.9
t12_m = 3.4

params_base = {"bl": 2e0, 
    "br": 6e0, 
    "dl": 2e-3, 
    "dr": 2e-3,
    "kb": 2e-4,
    "ku": 2e2, 
    "kp": 5.,
    "ks2": 1.,
    "kr": 1e-4,
    'krl':0.,
    'km':1e3,
    "ki": 1.5e1, 
    "ks3": 1e-1, 
    "ds": 8e-3,
    "n":1,
    "m":1,
    "KG": 6.6 * 0.1 * 1200., 
    "t12_G": t12_G,
    "t12_m": t12_m,
    "dG": np.log(2) / (60. * t12_G),
    "dm": np.log(2) / (60. * t12_m),
    "E0": 7e2,
    "TN": 1/60.,
    "CN": 500.} 

t0 = 0.
tf = 25. * 60.
time_l = [t0,tf]

c_max_11 = 0.68
c_max_41 = 0.59

c_max_44 = 1.4
c_max_14 = 3.6



In [ ]:
filename_0 = 'Fig3'
dir = os.getcwd() + '/' + filename_0
if os.path.isdir(dir) == False:
    os.makedirs(dir)
os.chdir(dir)


# Panel A

In [ ]:

params = dict(params_base)
k_i = np.logspace(-2,1.5,200)
p_p_names = ["bl"]
p_p_values=[k_i]

p_p_values_iter = list(itertools.product(*p_p_values))
p_p_values_iter.sort(key=lambda a: a[0])

param_iter = []

for  e in p_p_values_iter:
    param_i = dict(params)
    param_i.update(zip(p_p_names, e))
    param_iter.append(tuple([dict(param_i)]))

x_0 = [[round(params["br"]/params["dr"],2), 
       round(e[0]/params["dl"],2),
       round(params["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in p_p_values_iter]


In [ ]:

partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)
sol = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]


In [ ]:

cit_stat = [e.y[9][-1] for e in sol]
cit_max = [max(e.y[9]) for e in sol]

T_H = [e.t/60. for e in sol]


In [ ]:
values = [[k_i[74],k_i[102],k_i[159]],[cit_stat[74],cit_stat[102],cit_stat[159]]]
values

In [ ]:

fig,ax = plt.subplots()

ax.plot(k_i, cit_stat,color = cm_variables[0])
ax.plot(k_i, cit_max,color = cm_variables[0], ls = '--')

llss = ['-','--']
labels2 = [r"$Cit_{Stat}$", r"$Cit_{Max}$"]
h2 = [plt.plot([],[], color='#161616', ls=llss[j])[0] for j in range(len(llss))]
leg2 = ax.legend(handles=h2, labels=labels2, fontsize = 12, facecolor='white', framealpha=1, loc = (0.036,0.81))

ax.add_artist(leg2)

ax.set_box_aspect(0.7)
ax.set_xscale('log')
ax.set_xlabel(r'$\beta_L\ [min^{-1}]$')
ax.set_ylabel(r'Cit [a.u.]')

axes_tick_formatter(ax, 150, 75, axis = 'y')

for i in range(len(values[0])):
    ax.scatter(values[0][i],values[1][i], s=80, 
               facecolors='white', edgecolors=cm_variables[i])

ax.text(0.8e-1,200,'1')
ax.text(1,220,'2')
ax.text(7,165,'3')

fig.savefig(filename_0 + '_A.png', bbox_inches="tight", dpi = 300, transparent = True)


# Panel B

In [ ]:

fig,ax = plt.subplots()

labels1 = [1,2,3]

for i,j in enumerate([74,102,159]):
    ax.plot(T_H[j], sol[j].y[9],color = cm_variables[i])
    ax.plot(T_H[j], sol[j].y[7],color = cm_variables[i], ls = '--')

llss = ['-','--']
labels2 = [r"Cit", r"S"]
h2 = [plt.plot([],[], color='#161616', ls=llss[j])[0] for j in range(len(llss))]
leg2 = ax.legend(handles=h2, labels=labels2, fontsize = 12, facecolor='white', framealpha=1, loc = (0.8,0.83))

h1 = [plt.plot([],[], color=cm_variables[j], ls=llss[0])[0] for j in range(len(labels1))]
leg1= ax.legend(handles=h1, labels=labels1, fontsize = 12, facecolor='white', framealpha=1, loc = (0.825,0.6))

ax.set_ylabel("Cit, S" + ' [a.u.]')
ax.set_xlabel('t [h]')

ax.add_artist(leg1)
ax.add_artist(leg2)

axes_tick_formatter(ax, 150, 75, axis = 'y')
axes_tick_formatter(ax, 5, 2.5)

ax.set_ylim([None,620])

fig.savefig(filename_0 + '_B.png', bbox_inches="tight", dpi = 300, transparent = True)


# Panel C

In [ ]:

params = dict(params_base)
params.update({'kr':2e-3})
k_i = np.logspace(-3,1.5,200)
p_p_names = ["bl"]
p_p_values=[k_i]

p_p_values_iter = list(itertools.product(*p_p_values))
p_p_values_iter.sort(key=lambda a: a[0])

param_iter = []

for  e in p_p_values_iter:
    param_i = dict(params)
    param_i.update(zip(p_p_names, e))
    param_iter.append(tuple([dict(param_i)]))

x_0 = [[round(params["br"]/params["dr"],2), 
       round(e[0]/params["dl"],2),
       round(params["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in p_p_values_iter]


In [ ]:

partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)
sol = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]


In [ ]:

cit_stat = [e.y[9][-1] for e in sol]
cit_max = [max(e.y[9]) for e in sol]

T_H = [e.t/60. for e in sol]


In [ ]:
values = [[k_i[95],k_i[130],k_i[167]],[cit_max[95],cit_max[130],cit_max[167]]]
values

In [ ]:

fig,ax = plt.subplots()

ax.plot(k_i, cit_stat,color = cm_variables[1])
ax.plot(k_i, cit_max,color = cm_variables[1], ls = '--')

llss = ['-','--']
labels2 = [r"$Cit_{Stat}$", r"$Cit_{Max}$"]
h2 = [plt.plot([],[], color='#161616', ls=llss[j])[0] for j in range(len(llss))]
leg2 = ax.legend(handles=h2, labels=labels2, fontsize = 12, facecolor='white', framealpha=1, loc = (0.036,0.81))

ax.add_artist(leg2)

ax.set_box_aspect(0.7)
ax.set_xscale('log')
ax.set_xlabel(r'$\beta_L\ [min^{-1}]$')
ax.set_ylabel(r'Cit [a.u.]')

axes_tick_formatter(ax, 150, 75, axis = 'y')

for i in range(len(values[0])):
    ax.scatter(values[0][i],values[1][i], s=80, 
               facecolors='white', edgecolors=cm_variables[i])

ax.text(0.8e-1,180,'1')
ax.text(1.3,600,'2')
ax.text(7,825,'3')

fig.savefig(filename_0 + '_C.png', bbox_inches="tight", dpi = 300, transparent = True)


# Panel D

In [ ]:

fig,ax = plt.subplots()

labels1 = [1,2,3]

for i,j in enumerate([95,130,167]):
    ax.plot(T_H[j], sol[j].y[9],color = cm_variables[i])
    ax.plot(T_H[j], sol[j].y[7],color = cm_variables[i], ls = '--')

llss = ['-','--']
labels2 = [r"Cit", r"S"]
h2 = [plt.plot([],[], color='#161616', ls=llss[j])[0] for j in range(len(llss))]
leg2 = ax.legend(handles=h2, labels=labels2, fontsize = 12, facecolor='white', framealpha=1, loc = (0.805,0.71))

h1 = [plt.plot([],[], color=cm_variables[j], ls=llss[0])[0] for j in range(len(labels1))]
leg1= ax.legend(handles=h1, labels=labels1, fontsize = 12, facecolor='white', framealpha=1, loc = (0.83,0.42))

ax.set_ylabel("Cit, S" + ' [a.u.]')
ax.set_xlabel('t [h]')

ax.add_artist(leg1)
ax.add_artist(leg2)

axes_tick_formatter(ax, 200, 100, axis = 'y')
axes_tick_formatter(ax, 5, 2.5)

# ax.set_ylim([None,620])

fig.savefig(filename_0 + '_D.png', bbox_inches="tight", dpi = 300, transparent = True)


# Panel E

In [ ]:

params = dict(params_base)
params.update({'kr':2e-3, 'E0':5e3})

k_i = np.logspace(-4,1,6)
k_j = np.logspace(-4,5,200)

p_p_values = [k_i, k_j]
p_p_names = ["ks2","bl"]

p_p_values_iter = list(itertools.product(*p_p_values))
p_p_values_iter.sort(key=lambda a: a[0])


param_iter = []

for  e in p_p_values_iter:
    param_i = dict(params)
    param_i.update(zip(p_p_names, e))
    param_iter.append(tuple([dict(param_i)]))

x_0 = [[round(params["br"]/params["dr"],2), 
       round(e[1]/params["dl"],2),
       round(params["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in p_p_values_iter]


In [ ]:

partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)
sol = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]

[e[0].update({'kp':0}) for e in param_iter]
sol_kp0 = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]


In [ ]:

vv = 9
cit_stat = [e.y[vv][-1] for e in sol]
cit_stat_kp0 = [e.y[vv][-1] for e in sol_kp0]


In [ ]:

fig,ax = plt.subplots()

for i in range(len(k_i)):
    ax.plot(k_j, cit_stat[i*len(k_j):(i+1)*len(k_j)],color = cm_variables[i])
    ax.plot(k_j, cit_stat_kp0[i*len(k_j):(i+1)*len(k_j)],color = cm_variables[i], ls = '--')

llss = ['-','--']
labels2 = [r"$k_P = 5$", r"$k_P = 0$"]
h2 = [plt.plot([],[], color='#161616', ls=llss[j])[0] for j in range(len(llss))]
leg2 = ax.legend(handles=h2, labels=labels2, fontsize = 12, facecolor='white', framealpha=1, loc = (0.036,0.81))

labels1 = [r"$k_{S2} = $" + sci_notation(k_i[i]) for i in range(len(k_i))]
h1 = [plt.plot([],[], color=cm_variables[j], ls=llss[0])[0] for j in range(len(k_i))]
leg1= ax.legend(handles=h1, labels=labels1, fontsize = 12, facecolor='white', framealpha=1, loc = (0.036,0.30))   

ax.add_artist(leg1)
ax.add_artist(leg2)

ax.set_box_aspect(0.7)
ax.set_xscale('log')
ax.set_xlabel(r'$\beta_L\ [min^{-1}]$')
ax.set_ylabel(variables[vv] + r'$_{Stat}$ [a.u.]')


axes_tick_formatter(ax, 300, 150, axis = 'y')


x_major = mpl.ticker.LogLocator(base = 10.0, numticks = 5)
ax.xaxis.set_major_locator(x_major)
x_minor = mpl.ticker.LogLocator(base = 10.0, subs = np.arange(1.0, 10.0) * 0.1, numticks = 10)
ax.xaxis.set_minor_locator(x_minor)
ax.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())

fig.savefig(filename_0 + '_E.png', bbox_inches="tight", dpi = 300, transparent = True)


# Panel F

In [ ]:

params = dict(params_base)
params.update({'kr':2e-3, 'ks2':1e-3, 'E0': 5e3})

k_i = np.logspace(-6,-2,5)
k_j = np.logspace(-4,5,200)

p_p_values = [k_i, k_j]
p_p_names = ["kb","bl"]

p_p_values_iter = list(itertools.product(*p_p_values))
p_p_values_iter.sort(key=lambda a: a[0])


param_iter = []

for  e in p_p_values_iter:
    param_i = dict(params)
    param_i.update(zip(p_p_names, e))
    param_iter.append(tuple([dict(param_i)]))

x_0 = [[round(params["br"]/params["dr"],2), 
       round(e[1]/params["dl"],2),
       round(params["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in p_p_values_iter]


In [ ]:
partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)
sol = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]

vv = 9

cit_stat = [e.y[vv][-1] for e in sol]


[e[0].update({'kp':0}) for e in param_iter]
sol_kp0 = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]
cit_stat_kp0 = [e.y[vv][-1] for e in sol_kp0]


In [ ]:
[e[0].update({'kp':5, 'ks2':1.}) for e in param_iter]
sol_inset = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]
cit_stat_inset = [e.y[vv][-1] for e in sol_inset]

[e[0].update({'kp':0}) for e in param_iter]
sol_kp0_inset = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]
cit_stat_kp0_inset = [e.y[vv][-1] for e in sol_kp0_inset]


In [ ]:
fig,ax = plt.subplots()

for i in range(len(k_i)):
    ax.plot(k_j, cit_stat[i*len(k_j):(i+1)*len(k_j)],color = cm_variables[i])
    ax.plot(k_j, cit_stat_kp0[i*len(k_j):(i+1)*len(k_j)],color = cm_variables[i], ls = '--')

llss = ['-','--']
labels2 = [r"$k_P = 5$", r"$k_P = 0$"]
h2 = [plt.plot([],[], color='#161616', ls=llss[j])[0] for j in range(len(llss))]
leg2 = ax.legend(handles=h2, labels=labels2, fontsize = 12, facecolor='white', framealpha=1, loc = (0.036,0.81))

labels1 = [r"$k_{B} = $" + sci_notation(k_i[i]) for i in range(len(k_i))]
h1 = [plt.plot([],[], color=cm_variables[j], ls=llss[0])[0] for j in range(len(k_i))]
leg1= ax.legend(handles=h1, labels=labels1, fontsize = 12, facecolor='white', framealpha=1, loc = (0.036,0.38))   

ax.add_artist(leg1)
ax.add_artist(leg2)

ax.set_box_aspect(0.7)
ax.set_xscale('log')
ax.set_xlabel(r'$\beta_L\ [min^{-1}]$')
ax.set_ylabel(variables[vv] + r'$_{Stat}$ [a.u.]')

axes_tick_formatter(ax, 300, 150, axis = 'y')

x_major = mpl.ticker.LogLocator(base = 10.0, numticks = 5)
ax.xaxis.set_major_locator(x_major)
x_minor = mpl.ticker.LogLocator(base = 10.0, subs = np.arange(1.0, 10.0) * 0.1, numticks = 10)
ax.xaxis.set_minor_locator(x_minor)
ax.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())

zoom_ax = fig.add_axes([0,0,1,1])
zoom_ax.set_axes_locator(InsetPosition(ax, [0.69, 0.015,0.3,0.3]))

for i in range(len(k_i)):
    zoom_ax.plot(k_j, cit_stat_inset[i*len(k_j):(i+1)*len(k_j)],color = cm_variables[i])
    zoom_ax.plot(k_j, cit_stat_kp0_inset[i*len(k_j):(i+1)*len(k_j)],color = cm_variables[i], ls = '--')

zoom_ax.axes.get_xaxis().set_visible(False)
zoom_ax.axes.get_yaxis().set_visible(False)

zoom_ax.set_xscale('log')

fig.savefig(filename_0  + '_F.png', bbox_inches="tight", dpi = 300, transparent = True)
